<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 00 - Scenario Overview and System Setup

In this set of tutorials you'll work with an end-to-end scenario that uses SQL Server 2019's big data clusters to solve real-world problems. 


## Wide World Importers

**TODO:** Describe company and use-case

## Running these Tutorials

 - You'll need a SQL Server 2019 big data cluster installed to work through these tutorials, or you can just look at the output of these completed tutorials. 

- To set up your own cluster, check this location and follow the steps you see there. **TODO:** link to install

- You can follow along with the steps you see in these tutorials by copying what you see into a SQL Query window and Spark Notebook using the Azure Data Studio tool. 

- You can also download these Jupyter Notebooks and run them in Azure Data Studio for a more interactive experience.

- For a complete workshop on SQL Server 2019's big data clusters, check out this resource. **TODO:** link to bdc ws

## Copy Files to the SQL Server 2019 big data cluster

**TODO:** describe curl, etc. Code for file transfer

In [2]:
!md c:\temp
!dir c:\temp

## Next Step: Working with the SQL Server 2019 big data cluster Master Instance

**TODO:** Add Link